### Objetivo:
Para construir pipelines de dados robustos, é essencial dominar a interação com APIs REST. Isso inclui não apenas ler dados (`GET`), mas também criar (`POST`), atualizar (`PUT`/`PATCH`) e excluir (`DELETE`) recursos.

Este notebook serve como um "playground" para estudar e documentar o uso dos principais métodos HTTP contra a API de testes GoRest. O objetivo é simular o ciclo de vida completo de um usuário (criação, atualização e exclusão) de forma dinâmica e segura, capturando IDs e lidando com as regras da API.

In [ ]:
# Configurações iniciais.
import requests 
import os 
import time 
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
TOKEN = os.getenv("API_TOKEN")
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
URL = "https://gorest.co.in/public/v2"

In [4]:
# MÉTODO POST - Criação de usuários. 
PAYLOAD = [{"name": "Amanda",
           "email": "amanda@github.com",
           "gender": "female",
           "status": "inactive"}, # Adicionando status como 'inactive'.

            {"name": "ReNam", # Adicionando nome escrito de forma incorreta.
           "email": "renan@github.com",
           "gender": "male",
           "status": "inactive"}, # Adicionando status como 'inactive'.

           {"name": "Ana",
            "email": "ana@github.com",
            "gender": "female",
            "status": "active"},

            {"name": "Nelson",
             "email": "nelson@github.com",
             "gender": "male",
             "status": "active"}]

timestamp = int(time.time()) # Geração de valor único para garantir e=mails não se repitam (devido regra da API que proíbe e-mails duplicados).
ids = []

for user in PAYLOAD:
    user['email'] = f"{timestamp}-{user['email']}"
    
    response = requests.post(f"{URL}/users", json=user, headers=HEADERS)
    response.raise_for_status()

    if response.status_code == 201: 
        new_user = response.json()
        user_id = new_user['id']
        ids.append(user_id)
        print(f"Usuário {user['name']} criado com sucesso! ID: {user_id}")
    else: 
        print(f"Falha ao criar o usuário {user['name']}. Status Code: {response.status_code} - {response.json()}")

Usuário Amanda criado com sucesso! ID: 8324991
Usuário ReNam criado com sucesso! ID: 8324992
Usuário Ana criado com sucesso! ID: 8324993
Usuário Nelson criado com sucesso! ID: 8324994


In [5]:
# MÉTODO GET - VERIFICAÇÃO RÁPIDA    
response = requests.get(f"{URL}/users", headers=HEADERS)
df = pd.DataFrame(response.json())
df.head()

,id,name,email,gender,status
0,8324994,Nelson,1767920707-nelson@github.com,male,active
1,8324993,Ana,1767920707-ana@github.com,female,active
2,8324992,ReNam,1767920707-renan@github.com,male,inactive
3,8324991,Amanda,1767920707-amanda@github.com,female,inactive
4,8324990,Nelson,1767920624-nelson@github.com,male,active


In [6]:
# MÉTODO PATCH - Atualização de um único campo (Alteração de status). 
user_id = ids[0]
PAYLOAD_PATCH = {"status": "active"} 

response = requests.patch(f"{URL}/users/{user_id}", json=PAYLOAD_PATCH, headers=HEADERS)
response.raise_for_status()
if response.status_code == 200:
    print(f"Usuário com o ID {user_id} atualizado com sucesso!")
response = requests.get(f"{URL}/users", headers=HEADERS)
df = pd.DataFrame(response.json())
df.head()

Usuário com o ID 8324991 atualizado com sucesso!


,id,name,email,gender,status
0,8324991,Amanda,1767920707-amanda@github.com,female,active
1,8324994,Nelson,1767920707-nelson@github.com,male,active
2,8324993,Ana,1767920707-ana@github.com,female,active
3,8324992,ReNam,1767920707-renan@github.com,male,inactive
4,8324990,Nelson,1767920624-nelson@github.com,male,active


In [7]:
# MÉTODO PUT - Substituição completa de usuário (Correção de nome e status). 
user_id = ids[1]
PAYLOAD_PUT = {"name": "Renan", # Nome corrigido.
           "email": f"{timestamp}-renan@github.com",
           "gender": "male",
           "status": "active"} # Status corrigido.

response = requests.put(f"{URL}/users/{user_id}", json=PAYLOAD_PUT, headers=HEADERS)
response.raise_for_status()
if response.status_code == 200: 
    print(f"Usuário com o ID {user_id} atualizado com sucesso!")

Usuário com o ID 8324992 atualizado com sucesso!


In [8]:
# MÉTODO DELETE - Exclusão de todos os usuários. 
for user_id in ids: 
    response = requests.delete(f"{URL}/users/{user_id}", headers=HEADERS)
    response.raise_for_status()
    if response.status_code == 204:
        print(f"Usuário {user_id} excluídos com sucesso!")

Usuário 8324991 excluídos com sucesso!
Usuário 8324992 excluídos com sucesso!
Usuário 8324993 excluídos com sucesso!
Usuário 8324994 excluídos com sucesso!


### Conclusão
Através dos testes acima, foi validado o funcionamento de todo o ciclo CRUD (Create, Read, Update, Delete) na API GoRest. A captura dinâmica de IDs e a manipulação de e-mails únicos foram essenciais para criar um script robusto e reutilizável. O conhecimento adquirido aqui é a base para a construção da pipeline de dados automatizada